In [1]:
import pandas as pd
import numpy as np


In [10]:
raw0 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=0)
raw1 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=1)
raw2 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=2)
raw3 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=3)
raw4 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=4)
raw5 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=5)
raw6 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=6)
raw7 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=7)
raw8 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=8)
raw9 = pd.read_excel('../data/raw/safety_stock_sample.xlsx',sheet_name=9)

In [20]:
raw = raw0.append(raw1).append(raw2).append(raw3).append(raw4).append(raw4).append(raw6).append(raw7).append(raw8).append(raw9).reset_index(drop = True)

/var/folders/w4/8j97c539713d5s_5j6kdbrq00000gn/T/ipykernel_73966/1808302562.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw = raw0.append(raw1).append(raw2).append(raw3).append(raw4).append(raw4).append(raw6).append(raw7).append(raw8).append(raw9).reset_index(drop = True)
/var/folders/w4/8j97c539713d5s_5j6kdbrq00000gn/T/ipykernel_73966/1808302562.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw = raw0.append(raw1).append(raw2).append(raw3).append(raw4).append(raw4).append(raw6).append(raw7).append(raw8).append(raw9).reset_index(drop = True)
/var/folders/w4/8j97c539713d5s_5j6kdbrq00000gn/T/ipykernel_73966/1808302562.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw = raw0.append(raw1).append(raw2).append

In [21]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24825 entries, 0 to 24824
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Item number     24825 non-null  object        
 1   Physical date   24825 non-null  datetime64[ns]
 2   Financial date  24823 non-null  datetime64[ns]
 3   Quantity        24825 non-null  int64         
 4   Type            24825 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 969.9+ KB


In [23]:
raw.Type.value_counts()

Profit/Loss       24523
Purchase order      271
Transfer             30
Counting              1
Name: Type, dtype: int64

In [24]:
raw.head()

,Item number,Physical date,Financial date,Quantity,Type
0,50040800100,2022-01-01,2022-01-01,-10,Profit/Loss
1,50040800100,2022-01-01,2022-01-01,-10,Profit/Loss
2,50040800100,2022-01-01,2022-01-01,-12,Profit/Loss
3,50040800100,2022-01-01,2022-01-01,-3,Profit/Loss
4,50040800100,2022-01-01,2022-01-01,-31,Profit/Loss


In [26]:
# we only care about the daily consumption
raw = raw[(raw.Quantity <= 0)&(raw.Type == 'Profit/Loss')]

In [28]:
raw.head()

,Item number,Physical date,Financial date,Quantity,Type
0,50040800100,2022-01-01,2022-01-01,-10,Profit/Loss
1,50040800100,2022-01-01,2022-01-01,-10,Profit/Loss
2,50040800100,2022-01-01,2022-01-01,-12,Profit/Loss
3,50040800100,2022-01-01,2022-01-01,-3,Profit/Loss
4,50040800100,2022-01-01,2022-01-01,-31,Profit/Loss


In [29]:
# we want data at item numer and physical date level
raw['Quantity'] = -raw['Quantity']
raw = raw.rename(columns = {
    'Item number': 'item_number',
    'Physical date': 'date',
    'Quantity': 'consumption'
})

/var/folders/w4/8j97c539713d5s_5j6kdbrq00000gn/T/ipykernel_73966/338180742.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['Quantity'] = -raw['Quantity']


In [30]:
raw.head()

,item_number,date,Financial date,consumption,Type
0,50040800100,2022-01-01,2022-01-01,10,Profit/Loss
1,50040800100,2022-01-01,2022-01-01,10,Profit/Loss
2,50040800100,2022-01-01,2022-01-01,12,Profit/Loss
3,50040800100,2022-01-01,2022-01-01,3,Profit/Loss
4,50040800100,2022-01-01,2022-01-01,31,Profit/Loss


In [42]:
process_data = raw.groupby(['item_number', 'date']).consumption.sum().reset_index()

In [43]:
process_data.head()

,item_number,date,consumption
0,50040800100,2022-01-01,170
1,50040800100,2022-01-03,172
2,50040800100,2022-01-05,188
3,50040800100,2022-01-07,228
4,50040800100,2022-01-09,171


In [44]:
process_data.to_csv('../data/ingested/processed_data.csv', index = False)

In [45]:
process_data = pd.read_csv('../data/ingested/processed_data.csv')
process_data.head()

,item_number,date,consumption
0,50040800100,2022-01-01,170
1,50040800100,2022-01-03,172
2,50040800100,2022-01-05,188
3,50040800100,2022-01-07,228
4,50040800100,2022-01-09,171
